In [1]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
from urllib.parse import unquote

server_name   = "localhost"
database_name = "Lab2Joel"
username = 'sa'
password='reallyStrongPwd123'


connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};UID={username};PWD={password}"
url_string = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try:    
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=Lab2Joel;UID=sa;PWD=reallyStrongPwd123 

Successfully connected to Lab2Joel!


In [11]:
def search_books(search_term, connection):
    query = text("""
    SELECT b.Title, inv.StoreID, st.StoreName, inv.QuantityAvailable
    FROM Books b
    JOIN Inventory inv ON b.ISBN13 = inv.ISBN13
    JOIN Stores st ON inv.StoreID = st.ID
    WHERE b.Title LIKE :search_term;
    """)
    results = connection.execute(query, {'search_term': f'%{search_term}%'}).fetchall()

    return results

search_term = input("Search for title: ").strip()

if search_term:
    with engine.connect() as connection:
        results = search_books(search_term, connection)
        for result in results:
            print(f"Book Title: {result.Title}, is available in the store {result.StoreName} with {result.QuantityAvailable} books left")
else:
    print("No search title written. Terminating program")


Book Title: The Stand, is available in the store The Reading Nook with 16 books left
